In [1]:
import pandas as pd

df = pd.read_csv('pizza_sales_new.csv')

In [3]:
df.head()

,pizza_id,order_id,pizza_name_id,quantity,order_date,order_time,unit_price,total_price,pizza_size,pizza_category,pizza_ingredients,pizza_name
0,1,1,hawaiian_m,1,2015-01-01,11:38:36,13.25,13.25,M,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese",The Hawaiian Pizza
1,2,2,classic_dlx_m,1,2015-01-01,11:57:40,16.00,16.00,M,Classic,"Pepperoni, Mushrooms, Red Onions, Red Peppers,...",The Classic Deluxe Pizza
2,3,2,five_cheese_l,1,2015-01-01,11:57:40,18.50,18.50,L,Veggie,"Mozzarella Cheese, Provolone Cheese, Smoked Go...",The Five Cheese Pizza
3,4,2,ital_supr_l,1,2015-01-01,11:57:40,20.75,20.75,L,Supreme,"Calabrese Salami, Capocollo, Tomatoes, Red Oni...",The Italian Supreme Pizza
4,5,2,mexicana_m,1,2015-01-01,11:57:40,16.00,16.00,M,Veggie,"Tomatoes, Red Peppers, Jalapeno Peppers, Red O...",The Mexicana Pizza


In [7]:
df.columns

Index(['pizza_id', 'order_id', 'pizza_name_id', 'quantity', 'order_date',
       'order_time', 'unit_price', 'total_price', 'pizza_size',
       'pizza_category', 'pizza_ingredients', 'pizza_name'],
      dtype='object')

In [26]:
filtered_columns = ['pizza_name_id', 'quantity', 'order_date', 'order_time', 'pizza_category']

df_filtered = df[filtered_columns]

In [27]:
df_filtered.head()

,pizza_name_id,quantity,order_date,order_time,pizza_category
0,hawaiian_m,1,2015-01-01,11:38:36,Classic
1,classic_dlx_m,1,2015-01-01,11:57:40,Classic
2,five_cheese_l,1,2015-01-01,11:57:40,Veggie
3,ital_supr_l,1,2015-01-01,11:57:40,Supreme
4,mexicana_m,1,2015-01-01,11:57:40,Veggie


In [28]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48554 entries, 0 to 48553
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   pizza_name_id   48554 non-null  object
 1   quantity        48554 non-null  int64 
 2   order_date      48554 non-null  object
 3   order_time      48554 non-null  object
 4   pizza_category  48554 non-null  object
dtypes: int64(1), object(4)
memory usage: 1.9+ MB


In [29]:
df_filtered['order_date'] = pd.to_datetime(df_filtered['order_date'])

C:\Users\New\AppData\Local\Temp\ipykernel_16908\2778367975.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['order_date'] = pd.to_datetime(df_filtered['order_date'])


In [32]:
df_filtered.head()

,pizza_name_id,quantity,order_date,pizza_category
0,hawaiian_m,1,2015-01-01,Classic
1,classic_dlx_m,1,2015-01-01,Classic
2,five_cheese_l,1,2015-01-01,Veggie
3,ital_supr_l,1,2015-01-01,Supreme
4,mexicana_m,1,2015-01-01,Veggie


In [31]:
df_filtered.drop(columns=['order_time'], inplace=True)

C:\Users\New\AppData\Local\Temp\ipykernel_16908\617353525.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.drop(columns=['order_time'], inplace=True)


In [33]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48554 entries, 0 to 48553
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   pizza_name_id   48554 non-null  object        
 1   quantity        48554 non-null  int64         
 2   order_date      48554 non-null  datetime64[ns]
 3   pizza_category  48554 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 1.5+ MB


In [34]:
df_filtered.to_csv('pizza_sales_final.csv', index=False)

In [40]:
df_sales = df_filtered.groupby(['pizza_name_id','order_date'])['quantity'].sum().reset_index()

In [42]:
df_sales.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22945 entries, 0 to 22944
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   pizza_name_id  22945 non-null  object        
 1   order_date     22945 non-null  datetime64[ns]
 2   quantity       22945 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 537.9+ KB


In [50]:
from prophet import Prophet
import matplotlib.pyplot as plt
# Create an empty list to store forecast results
forecast_results = []

# Loop through each unique pizza_name_id
for pizza_id in df_sales['pizza_name_id'].unique():
    # Filter data for the current pizza
    df_pizza = df_sales[df_sales['pizza_name_id'] == pizza_id][['order_date', 'quantity']]
    df_pizza = df_pizza.rename(columns={'order_date': 'ds', 'quantity': 'y'})  # Prophet format

    # Check if the data is sufficient for training
    if len(df_pizza) < 10:  # Skip pizzas with very few sales records
        continue

    # Initialize and train Prophet model
    model = Prophet()
    model.fit(df_pizza)

    # Create future dates for prediction (next 7 days)
    future = model.make_future_dataframe(periods=7)

    # Forecast sales
    forecast = model.predict(future)

    # Keep only relevant columns
    forecast = forecast[['ds', 'yhat']].tail(7)  # Keep last 7 predictions
    forecast['pizza_name_id'] = pizza_id  # Add pizza identifier

    # Store in results list
    forecast_results.append(forecast)

# Combine all forecasts into a single dataframe
df_forecast_all = pd.concat(forecast_results, ignore_index=True)

# Display the forecasted sales for all pizzas
df_forecast_all.head()


11:46:50 - cmdstanpy - INFO - Chain [1] start processing
11:46:50 - cmdstanpy - INFO - Chain [1] done processing
11:46:50 - cmdstanpy - INFO - Chain [1] start processing
11:46:51 - cmdstanpy - INFO - Chain [1] done processing
11:46:51 - cmdstanpy - INFO - Chain [1] start processing
11:46:51 - cmdstanpy - INFO - Chain [1] done processing
11:46:51 - cmdstanpy - INFO - Chain [1] start processing
11:46:51 - cmdstanpy - INFO - Chain [1] done processing
11:46:51 - cmdstanpy - INFO - Chain [1] start processing
11:46:51 - cmdstanpy - INFO - Chain [1] done processing
11:46:51 - cmdstanpy - INFO - Chain [1] start processing
11:46:51 - cmdstanpy - INFO - Chain [1] done processing
11:46:51 - cmdstanpy - INFO - Chain [1] start processing
11:46:52 - cmdstanpy - INFO - Chain [1] done processing
11:46:52 - cmdstanpy - INFO - Chain [1] start processing
11:46:52 - cmdstanpy - INFO - Chain [1] done processing
11:46:52 - cmdstanpy - INFO - Chain [1] start processing
11:46:52 - cmdstanpy - INFO - Chain [1]

,ds,yhat,pizza_name_id
0,2016-01-01,3.289330,bbq_ckn_l
1,2016-01-02,3.459745,bbq_ckn_l
2,2016-01-03,2.647700,bbq_ckn_l
3,2016-01-04,2.820554,bbq_ckn_l
4,2016-01-05,2.591089,bbq_ckn_l


In [53]:
df_forecast_all['yhat'] = df_forecast_all['yhat'].round()


In [54]:
df_forecast_all.head()

,ds,yhat,pizza_name_id
0,2016-01-01,3.0,bbq_ckn_l
1,2016-01-02,3.0,bbq_ckn_l
2,2016-01-03,3.0,bbq_ckn_l
3,2016-01-04,3.0,bbq_ckn_l
4,2016-01-05,3.0,bbq_ckn_l


In [55]:
df_forecast_all

,ds,yhat,pizza_name_id
0,2016-01-01,3.0,bbq_ckn_l
1,2016-01-02,3.0,bbq_ckn_l
2,2016-01-03,3.0,bbq_ckn_l
3,2016-01-04,3.0,bbq_ckn_l
4,2016-01-05,3.0,bbq_ckn_l
...,...,...,...
632,2016-01-03,2.0,veggie_veg_s
633,2016-01-04,2.0,veggie_veg_s
634,2016-01-05,2.0,veggie_veg_s
635,2016-01-06,2.0,veggie_veg_s


In [56]:
df_forcast_sum = df_forecast_all.groupby(['pizza_name_id'])['yhat'].sum().reset_index()

In [57]:
df_forcast_sum

,pizza_name_id,yhat
0,bbq_ckn_l,21.0
1,bbq_ckn_m,20.0
2,bbq_ckn_s,14.0
3,big_meat_s,37.0
4,brie_carre_s,14.0
...,...,...
86,the_greek_xl,15.0
87,the_greek_xxl,7.0
88,veggie_veg_l,14.0
89,veggie_veg_m,14.0


In [58]:
df_forcast_sum.to_csv('pizza_sales_forecast.csv', index=False)